In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_promotion_free_gift_current
USING DELTA
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
SELECT 
  DATE_SUB(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), 1) AS calday, 
  store_id, 
  buy_item_id AS product_id, 
  concat_ws(', ', collect_list(DISTINCT CONCAT(bonus_buy_id, '-', bonus_buy_note))) AS bonus_buy_id
FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_promotion
WHERE bonus_buy_profile = 'ZB01'
  AND bonus_buy_status IS NULL
  AND DATE_SUB(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), 1) BETWEEN promo_start_date AND promo_end_date
  AND LEFT(bonus_buy_note, 1) = 'B'
GROUP BY 1,2,3
""")